# 합성곱 신경망 (CNN, convolutional neural network)

# 합성곱 계층

#### CNN vs 완전연결 계층
완전 연결 계층은 **데이터의 형상이 무시**된다.<br>
이미지는 가로, 세로, 채널로 이루어져있는 3차원 데이터이므로 인접한 픽셀끼리는 공간적 의미가 비슷할 수밖에 없다.<br>
CNN의 입출력 데이터를 **feature map**이라고 부른다.

### 합성곱 연산(필터 연산)
단일 곱셈-누산: 입력 feature map에서 윈도우를 일정한 간격으로 이동해가며 필터를 적용한다. 출력 featurm map이 생긴다.<br>
CNN의 필터 매개변수가 완전연결 계층의 가중치 매개변수, 편향의 역할을 한다.

### 패딩
덧대기<br>
입력 feature map의 주변에 0을 덧댄다. 주로 출력 shape을 조정할 목적으로 사용.<br>
합성곱 연산을 거듭하다가 shape이 1이 되는 것을 미연에 방지하기 위함

### 스트라이드
보폭, 윈도우가 이동하는 일정한 간격을 조정한다.<br>
출력의 크기가 줄어든다.

## 3차원 데이터의 필터 연산
(가로, 세로, 채널)의 3차원 **input feature map의 채널 수 = 필터의 채널 수**<br>
=> output feature map의 채널 = 1 <br>
**출력의 채널 수**는 **필터의 개수**로 조정한다.<br>
따라서 필터는 (필터 개수, 채널, 가로, 세로) 4차원 데이터

### 배치 처리
input feature map 여러 개를 한 번에 묶는다.
- feature map 형상: (데이터 수, 채널 수, 가로, 세로)
- filter 형상: (필터 수, 채널 수, 가로, 세로)

# 풀링 계층

풀링 윈도우가 스트라이드 만큼 이동하면서 윈도우 내부의 원소 중 최대 or 평균 원소를 출력한다. -> 가로, 세로를 축소시킴

# 7.4 합성곱/풀링 계층 구현하기

In [1]:
import numpy as np
x = np.random.rand(10,1,28,28) # (데이터 수, 채널 수, 가로, 세로)
x.shape

(10, 1, 28, 28)

In [2]:
x[0].shape # 첫 번째 데이터

(1, 28, 28)

In [3]:
x[0,0].shape # 첫 번째 데이터의 첫 번째 채널

(28, 28)

### im2col
이미지를 column으로

In [4]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화).
    
    Parameters
    ----------
    input_data : 4차원 배열 형태의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    col : 2차원 배열
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col

In [5]:
x1 = np.random.rand(1,3,7,7)
col1 = im2col(x1, 5,5,stride=1, pad=0)
print(col1.shape)

(9, 75)


In [6]:
x2 = np.random.rand(10,3,7,7)
col2 = im2col(x2, 5,5, stride=1, pad=0)
print(col2.shape)

(90, 75)


채널[0]의 원소들 + 채널[1]의 원소들 + 채널[2]의 원소들 = 75차원

In [7]:
# 합성곱 계층 구현하기
class Convoution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W=W
        self.b=b
        self.stride = stride
        self.pad=pad

    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1+(H+2*self.pad-FH)/self.stride)
        out_w = int(1+(W+2*self.pad-FW)/self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T # 필터 전개
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0,3,1,2)
        
        return out

- 풀링 계층 구현하기<br>
입력 데이터를 전개 -> 최댓값 나열 -> reshape

In [8]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
        self.x = None
        self.arg_max = None

    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)

        arg_max = np.argmax(col, axis=1)
        out = np.max(col, axis=1)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        self.x = x
        self.arg_max = arg_max

        return out

# 7.5 CNN 구현하기  

In [12]:
class SimpleConvNet:
    def __init__(self, input_dim=(1,28,28), 
                 conv_param={'filter_num': 30, 'filter_size':5, 'pad':0, 'stride':1}, 
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad)/filter_strdie + 1
        pool_output_size = int(filter_num*(conv_output_size/2)*(conv_output_size/2))

        # 매개변수 초기화
        self.params={}
        self.params['W1'] = weight_init_std*np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)

        # 역전파
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layer.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads={}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db